# Fator de Visão de Céu a partir do LiDAR

Podemos determinar o Fator de Visão de céu, ou Sky View Factor diretamente do LiDAR sem gerar um raster.
Para fins de performance podemos reduzir a densidade de pontos para cerca de 1 metro de resolução e a partir de cada ponto de Ground calcular a coordenada astronômica (Azimute e Altura) e a distância linear para cada ponto classificados como edificação.

Para esse primeiro experimento vamos apenas usar uma folha SCM, sabendo que inerferências para visão de céu podem se estender muito além dos limites das dimensões de uma folha SCM.

In [1]:
import pdal

In [2]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 5
    }
  ]
}"""

In [3]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
arrays = r.arrays

In [4]:
len(arrays[0])

28449

In [5]:
arrays[0][0]

(333499.29, 7395564.35, 735.34, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36606547)

In [6]:
r.schema

{'schema': {'dimensions': [{'name': 'X', 'size': 8, 'type': 'floating'},
   {'name': 'Y', 'size': 8, 'type': 'floating'},
   {'name': 'Z', 'size': 8, 'type': 'floating'},
   {'name': 'Intensity', 'size': 2, 'type': 'unsigned'},
   {'name': 'ReturnNumber', 'size': 1, 'type': 'unsigned'},
   {'name': 'NumberOfReturns', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanDirectionFlag', 'size': 1, 'type': 'unsigned'},
   {'name': 'EdgeOfFlightLine', 'size': 1, 'type': 'unsigned'},
   {'name': 'Classification', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanAngleRank', 'size': 4, 'type': 'floating'},
   {'name': 'UserData', 'size': 1, 'type': 'unsigned'},
   {'name': 'PointSourceId', 'size': 2, 'type': 'unsigned'},
   {'name': 'GpsTime', 'size': 8, 'type': 'floating'}]}}

## Importando dados para Pandas

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame(arrays[0])

In [17]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
0,333499.29,7395564.35,735.34,18,1,1,0,0,6,-14.0,17,6,356569.366065
1,333499.10,7395569.68,735.30,17,1,1,0,0,6,-15.0,17,6,356569.366153
2,333499.29,7395521.99,751.69,10,1,1,0,0,6,-11.0,17,6,356569.383594
3,333499.27,7395537.84,736.85,9,1,1,0,0,6,-12.0,17,6,356569.383810
4,333499.09,7395543.14,736.53,15,1,1,0,0,6,-13.0,17,6,356569.383898
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28444,333592.36,7395545.25,729.15,3,1,2,1,0,19,7.0,40,6,363043.632359
28445,333554.39,7395545.09,741.76,33,1,1,1,0,6,10.0,40,6,363043.651367
28446,333520.06,7395558.13,729.85,2,1,2,1,0,19,13.0,40,6,363044.033739
28447,333587.32,7395563.87,744.67,14,1,1,1,0,6,7.0,40,6,363044.068740


In [18]:
df.Classification.unique()

array([ 6, 20, 19,  2,  5,  9], dtype=uint8)

## Calculando o Azimute entre dois pontos

Suponhamos que fossemos calcular o azimute para cada ponto poderíamos fazer utilizando a seguinte função

In [20]:
import math

def Az(x1,y1,x2,y2):
    deg = math.degrees(math.atan2((x2 - x1),(y2 - y1)))
    if (deg < 0):
        deg += 360.0
    return deg

In [21]:
Az(1, 1, 1, 0)

180.0

## Calculando a Altura ou Elevação do ponto em relação ao Zenite

Da mesma forma poderíamos calcular a Altura/Elevação para cada um dos pontos usando a seguinte função

In [22]:
from scipy.spatial import distance

In [24]:
## Calculando altura
def altura_graus(x1, y1, z1, x2, y2, z2):
    altura = z2 - z1
    distancia = distance.euclidean([x1, y1], [x2, y2])
    return math.degrees(math.atan2(altura, distancia))

altura_graus(0,0,0,1,1,1)

35.264389682754654

## Gerando pontos x,y,z 

Apesar de termos diversas dimensões disponíveis para os calculos vamos nos ater em X, Y e Z e na Classificação para poder agrupar o que é Ground e o Que é edificação

In [25]:
df_ground = pd.DataFrame.from_records(df[df.Classification == 2][['X', 'Y', 'Z']].head(1000))

In [26]:
df_ground

,X,Y,Z
0,333475.91,7395545.19,724.75
1,333471.38,7395557.39,725.11
2,333465.71,7395556.51,725.17
3,333458.28,7395549.13,724.85
4,333454.93,7395553.02,724.84
...,...,...,...
978,333572.35,7395548.48,724.32
979,333589.63,7395543.64,724.11
980,333513.14,7395500.41,724.62
981,333513.53,7395510.17,724.71


In [28]:
df_building = pd.DataFrame.from_records(df[df.Classification == 6][['X', 'Y', 'Z']])
df_building = df_building.rename(columns={'X':'X1', 'Y':'Y1', 'Z':'Z1'})

In [29]:
df_building

,X1,Y1,Z1
0,333499.29,7395564.35,735.34
1,333499.10,7395569.68,735.30
2,333499.29,7395521.99,751.69
3,333499.27,7395537.84,736.85
4,333499.09,7395543.14,736.53
...,...,...,...
18830,333547.00,7395521.79,736.40
18831,333550.94,7395542.59,756.27
18832,333554.39,7395545.09,741.76
18833,333587.32,7395563.87,744.67


## Calculando o Azimute com Matriz de pontos

Portanto agora que temos os pontos de origem e os pontos alvo podemos calcular o Azimute. Porém se fizermos isso ponto a ponto não vamos usar os recursos computacionais de vetorização que promovem grande eficiência computacional.
Ainda não será exatamente essa função que vamos utilzar em produção, mas para fins didáticos e de aprendizado vamos mante-las.

In [30]:
import numpy as np

def np_az(nd_building, nd_ground):
    delta_x = nd_building[:, 0] - nd_ground[0, 0]
    delta_y = nd_building[:, 1] - nd_ground[0, 1]
    degree = np.arctan2(delta_x, delta_y) * 180 / np.pi
    degree[np.less(degree, 0)] += 360.0
    return degree

In [31]:
## Gerando dados de teste
test_ground = np.array([[0.,0.,0.], [1.,1.,1.]])
test_building = np.array([[0.,2.,2.], [2.,0,2.], [0.,-2.,2.], [-2.,0,18787.], [-2.,9899.,0.1]])

In [33]:
az = np_az(test_building, test_ground)
az

array([  0.        ,  90.        , 180.        , 270.        ,
       359.98842393])

## Calculando a altura/elevação com matriz de pontos

Da mesma forma vamos fazer com a altura/elevação

In [34]:
## Calculando altura/elevação

from scipy.spatial import distance

def np_altura(nd_building, nd_ground):
    altura = nd_building[:, 2] - nd_ground[0, 2]
    distancia = np.sqrt(np.sum((nd_building[:, [0,1]] - nd_ground[0, [0,1]])**2, axis=1))
    return np.arctan2((nd_building[:, 2] - nd_ground[0, 2]), distancia) * 180 / np.pi

In [35]:
altura = np_altura(test_building, test_ground)
altura

array([4.50000000e+01, 4.50000000e+01, 4.50000000e+01, 8.99939005e+01,
       5.78803701e-04])

In [36]:
nd_coordenadas = np.array([az, altura])
nd_coordenadas.transpose().shape

(5, 2)

## Utilizando os dados de Building

As funcões de calculo de Azimute e Altura/Elevação produzidas até agora aqui nesse NoteBook precisam ser iteradas sobre cada linha de ponto de ground. Portanto precisamos refatorar essas funções para trabalhar e iterar sobre cada um dos pontos utilizando o potencial computacional do NumPy, ou seja, trabalhando com vetorização.

In [38]:
#nd_building = df_building.to_numpy()
#nd_ground = df_ground.to_numpy()

In [39]:
#def calc_svf(nd_building, nd_ground):
#    az = np_az(nd_building, nd_ground)
#    altura = np_altura(nd_building, nd_ground)
#    coordenadas = np.array([az, altura])
#    return coordenadas.transpose()

In [43]:
#calc_svf(nd_building, nd_ground)

### Refatorando as funções de Azimute e Altura para trabalhar com o vetorização de Ground e Building

Para fins computacionais e menos iterações de laço que tendem a não ser tão eficientes, vamos tentar usar os recursos de vetorização e evitar o laço repetitivo

In [93]:
def np_coordenadas(nd_building, nd_ground):
    n2d_ground_z = nd_ground[:, 2].reshape(-1, 1)
    n2dt_building_z = nd_building[:, 2].reshape(-1, 1).transpose()
    
    nd_altura = np.subtract(n2dt_building_z, n2d_ground_z)
    
    n2d_ground_x = nd_ground[:, 0].reshape(-1, 1)
    n2dt_building_x = nd_building[:, 0].reshape(-1, 1).transpose()
    nd_delta_x = np.subtract(n2dt_building_x, n2d_ground_x)
    
    n2d_ground_y = nd_ground[:, 1].reshape(-1, 1)
    n2dt_building_y = nd_building[:, 1].reshape(-1, 1).transpose()
    nd_delta_y = np.subtract(n2dt_building_y, n2d_ground_y)
    
    nd_distancia = np.hypot(nd_delta_x, nd_delta_y)
    
    nd_elevacao = np.degrees(np.arctan2(nd_altura, nd_distancia))
    #nd_elevacao[np.less(nd_elevacao, 0.)] += 360.0
    
    nd_azimute = np.arctan2(nd_delta_x, nd_delta_y) * 180 / np.pi
    nd_azimute[np.less(nd_azimute, 0.)] += 360.0
    
    return nd_azimute, nd_elevacao

In [94]:
## Gerando dados de teste
test_ground = np.array([[0.,0.,0.], [-1.,-1.,-1.], [1.,1.,1.]], dtype='float32')
test_building = np.array([[0.,2.,2.], [2.,0,2.], [0.,-2.,2.], [-2.,0,18787.], [-2.,9899.,0.1]], dtype='float32')

In [159]:
%timeit azimute, elevacao = np_coordenadas(nd_building, nd_ground)
print("Distância: ")
print(elevacao.shape)
print(elevacao.size)
print("Elevacao: ")
print(azimute.shape)
print(azimute.size)

1.23 s ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Distância: 
(983, 18835)
18514805
Elevacao: 
(983, 18835)
18514805


In [140]:
nd_coordenadas = np.stack((azimute, elevacao), axis=1)

In [141]:
nd_coordenadas

array([[[ 0.0000000e+00,  9.0000000e+01,  1.8000000e+02,  2.7000000e+02,
          3.5998843e+02],
        [ 4.5000000e+01,  4.5000000e+01,  4.5000000e+01,  8.9993896e+01,
          5.7880371e-04]],

       [[ 1.8434948e+01,  7.1565048e+01,  1.3500000e+02,  3.1500000e+02,
          3.5999420e+02],
        [ 4.3491516e+01,  4.3491516e+01,  6.4760590e+01,  8.9995682e+01,
          6.3661975e-03]],

       [[ 3.1500000e+02,  1.3500000e+02,  1.9843495e+02,  2.5156505e+02,
          3.5998264e+02],
        [ 3.5264389e+01,  3.5264389e+01,  1.7548399e+01,  8.9990356e+01,
         -5.2097593e-03]]], dtype=float32)

### Menos linhas, mais legibilidade, talvez mais eficiência

Podemos ainda refatorar o código acima e reduzir algumas linhas de código, trabalhando desde o começo com 3 dimensões. Isso ainda pode acarretar alguma melhora na performance computacional mas não é a intenção por enquanto.

In [142]:
def np_coordenadas(nd_building, nd_ground):
    n2d_ground = nd_ground[:, :].reshape(-1, 3).astype('float32')
    n2d_ground = n2d_ground.reshape(-1, 3, 1)

    n2dt_building = nd_building[:, :].reshape(-1, 3).astype('float32')
    n2dt_building = n2dt_building.reshape(-1, 3, 1).transpose()

    nd_delta = np.subtract(n2dt_building, n2d_ground)
    
    nd_altura = nd_delta[:,2,:]
    
    nd_distancia = np.hypot(nd_delta[:,0,:], nd_delta[:,1,:])
    
    nd_elevacao = np.degrees(np.arctan2(nd_altura, nd_distancia))
    nd_elevacao[np.less(nd_elevacao, 0.)] += 360.0
    
    nd_azimute = np.arctan2(nd_delta[:,1,:], nd_delta[:,0,:]) * 180 / np.pi
    nd_azimute[np.less(nd_azimute, 0.)] += 360.0
    
    return nd_azimute, nd_elevacao

In [162]:
azimute, elevacao = np_coordenadas(test_building, test_ground)
print("Distância: ")
print(elevacao.shape)
print(elevacao.size)
print("Elevacao: ")
print(azimute.shape)
print(azimute.size)

Distância: 
(3, 5)
15
Elevacao: 
(3, 5)
15


In [163]:
nd_coordenadas = np.stack((azimute, elevacao), axis=1)

In [164]:
nd_coordenadas

array([[[9.0000000e+01, 0.0000000e+00, 2.7000000e+02, 1.8000000e+02,
         9.0011566e+01],
        [4.5000000e+01, 4.5000000e+01, 4.5000000e+01, 8.9993896e+01,
         5.7880371e-04]],

       [[7.1565048e+01, 1.8434948e+01, 3.1500000e+02, 1.3500000e+02,
         9.0005791e+01],
        [4.3491516e+01, 4.3491516e+01, 6.4760590e+01, 8.9995682e+01,
         6.3661975e-03]],

       [[1.3500000e+02, 3.1500000e+02, 2.5156505e+02, 1.9843495e+02,
         9.0017357e+01],
        [3.5264389e+01, 3.5264389e+01, 1.7548399e+01, 8.9990356e+01,
         3.5999478e+02]]], dtype=float32)

## Calculando o SVF

Agora para calcular o SVF vamos dividir a semi-esfera visível do céu em N fusos, definido pela variável`quantidade_de_fusos`.
E cada um dos fusos terá uma área esférica visível definido pela função `proporcao_visivel_no fuso`.
Depois de disponível essa função vamos iterar ela sobre todos os pontos de ground resultando no retorno da função `proporcao_visivel_total`.

In [168]:
quantidade_de_fusos = 36